# Generate Power Functional Connectivity
This script performs further processing on the data by generating the functional connectivities from the preprocessed data. We take in data from the final output of AA in the `aamod_firstlevel_modelestimate_saveresids_00001` folder and generate functional connectivity from the NIfTI files using the Power et al. brain atlas composed of 264 ROIs.

I'm following the guide here: <https://nilearn.github.io/auto_examples/03_connectivity/plot_sphere_based_connectome.html>

## Set up imports

In [1]:
from os import listdir, makedirs
from os.path import isfile, join, exists
import time

from nilearn import plotting, image, datasets
from nilearn.input_data import NiftiSpheresMasker
from nilearn.connectome import ConnectivityMeasure
import numpy as np

## Set up input and output

In [2]:
aa_folder = 'BioBank_Analysis/'
saveresids_folder = 'aamod_firstlevel_modelestimate_saveresids_00001/'
python_fc_folder = 'python_power_fc'

## Set up FC functions

In [3]:
power = datasets.fetch_coords_power_2011()
coords = np.vstack((power.rois['x'], power.rois['y'], power.rois['z'])).T

spheres_masker = NiftiSpheresMasker(
    seeds=coords, smoothing_fwhm=6, radius=5., standardize=True)

def generate_power_fc(file):
    time_series = spheres_masker.fit_transform(file)
    
    correlation_measure = ConnectivityMeasure(kind='correlation')
    correlation_matrix = correlation_measure.fit_transform([time_series])[0]
    
    return correlation_matrix

In [4]:
def visualize_corr_matrix(corr_matrix):
    np.fill_diagonal(corr_matrix, 0)
    plotting.plot_matrix(corr_matrix, vmin=-1., vmax=1., colorbar=True, figure=(20, 20),
                     title='Power correlation matrix')
#     plotting.plot_connectome(corr_matrix, coords, title='Power correlation graph',
#                          edge_threshold='98.0%', node_size=20, colorbar=True)

## Generate FCs

In [7]:
for age in range(7, 13):
    print("Processing age:", age)
    age_dir = f'/imaging/owenlab/klyons/fMRI/CBS_DevCog/BioBankData/Age{age}Data/'
    out_dir = f'/imaging3/owenlab/bpho/Kathleen_Data/Age{age}'

    input_path = join(age_dir, aa_folder, saveresids_folder)
    output_path = join(out_dir, python_fc_folder)

    # Grab all subjects
    subjects = [folder for folder in listdir(input_path) if folder.startswith("sub-")]

    for subject in subjects:
        print("  Processing subject:", subject)
        start_time = time.time()
        subject_input_path = join(input_path, subject)
        subject_output_path = join(output_path, subject)

        # Check if we've already generated the FC for this subject
        if exists(subject_output_path):
            print("  Skipping subject due to already generated FC.")
            continue

        subject_niftis = [join(subject_input_path, f) for f in listdir(subject_input_path) 
                          if isfile(join(subject_input_path, f)) and f.startswith('ResI')]

        if len(subject_niftis) != 745:
            print("  Incomplete subject niftis:", len(subject_niftis))
            continue

        subject_concat_image = image.concat_imgs(subject_niftis)
        subject_corr_matrix = generate_power_fc(subject_concat_image)

        makedirs(subject_output_path)
        subject_corr_filename = join(subject_output_path, 'power_fc.npy')
        np.save(subject_corr_filename, subject_corr_matrix)
        print("  Time elapsed:", time.time() - start_time)
    #     visualize_corr_matrix(subject_corr_matrix)

Processing age: 8
  Processing subject: sub-NDARFB908HVX
  Skipping subject due to already generated FC.
  Processing subject: sub-NDAREK549XUQ
  Incomplete subject niftis: 747
  Processing subject: sub-NDARJP444DZM
  Time elapsed: 115.79220485687256
  Processing subject: sub-NDARDE294HNX
  Time elapsed: 115.84988188743591
  Processing subject: sub-NDARBJ159HXB
  Time elapsed: 122.65610384941101
  Processing subject: sub-NDARBU928LV0
  Time elapsed: 128.0795841217041
  Processing subject: sub-NDARVP953BBF
  Incomplete subject niftis: 747
  Processing subject: sub-NDARMT784NED
  Time elapsed: 118.35569143295288
  Processing subject: sub-NDARPJ832WL6
  Incomplete subject niftis: 747
  Processing subject: sub-NDARUE603CHQ
  Time elapsed: 115.84178495407104
  Processing subject: sub-NDARMM951YEH
  Time elapsed: 116.08378624916077
  Processing subject: sub-NDARWV625CRP
  Incomplete subject niftis: 747
  Processing subject: sub-NDARRX819BZ0
  Time elapsed: 116.10409617424011
  Processing sub